In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import Levenshtein

In [ ]:
df = pd.read_csv('../datasets/DataSet_WebPhishing_Final.csv')
df_urls_legit = pd.read_csv('../datasets/top_websites.csv')

In [ ]:
# Adding a new column 'new_column' with default value 0
df['new_column'] = 0
df_urls_legit['new_column'] = 0

In [ ]:
# Selecting the first 5 rows and converting to a list
df_list = df.head(5).values.tolist()
df_urls_legit_list = df_urls_legit.head(5).values.tolist()